In [ ]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai

In [ ]:
import openai_config
import os

os.environ["OPENAI_API_KEY"] = openai_config.OPENAI_API_KEY

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artists LIMIT 10;")

sqlite
['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [ ]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_community.llms import OpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, verbose=True)
chain = create_sql_query_chain(llm, db)

In [ ]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [ ]:
print(db.get_table_info())


CREATE TABLE albums (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES artists ("ArtistId")
)

/*
3 rows from albums table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE artists (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from artists table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE customers (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES employe

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

def invoke_chain(user_question):
  sql_query = chain.invoke({"question": user_question})
  print("SQL query:", sql_query, end="\n\n")
  result = db.run(sql_query)
  print("Result:", result, end="\n\n")

  answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, generate a proper reply to give to user

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

  llm = LLMChain(llm=OpenAI(), prompt=answer_prompt)
  ans = llm(inputs={"question": user_question, "query": sql_query, "result": result})
  print("Response:",ans['text'])

In [ ]:
invoke_chain("How many employees are there currently?")

SQL query: SELECT COUNT("EmployeeId") AS "TotalEmployees" FROM employees;

Result: [(8,)]

Response: There are currently 8 employees. 


In [ ]:
invoke_chain("Where does Andrew Adams live?")

SQL query: SELECT "Address", "City", "State", "Country"
FROM employees
WHERE "FirstName" = 'Andrew' AND "LastName" = 'Adams'
LIMIT 1;

Result: [('11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada')]

Response: Andrew Adams lives at 11120 Jasper Ave NW, Edmonton, AB, Canada.


In [ ]:
invoke_chain("Give me the names of all the people who are from Canada and are working as Sales Manager")

SQL query: SELECT c."FirstName", c."LastName"
FROM customers c
JOIN employees e ON c."SupportRepId" = e."EmployeeId"
WHERE c."Country" = 'Canada' AND e."Title" = 'Sales Manager'
LIMIT 5;

Result: 

Response: 
Here are the names of five people from Canada who are working as Sales Manager:
- Sarah Johnson
- Michael Lee
- Emily Chen
- David Nguyen
- Olivia Kim 


In [ ]:
#Hallucination as no result was provided
invoke_chain("Give me Rock Songs")

SQL query: SELECT "tracks"."Name", "albums"."Title", "artists"."Name"
FROM "tracks"
JOIN "albums" ON "tracks"."AlbumId" = "albums"."AlbumId"
JOIN "artists" ON "albums"."ArtistId" = "artists"."ArtistId"
JOIN "genres" ON "tracks"."GenreId" = "genres"."GenreId"
WHERE "genres"."Name" = 'Rock'
LIMIT 5;

Result: [('AC/DC', 'For Those About To Rock We Salute You'), ('Accept', 'Balls to the Wall'), ('Accept', 'Restless and Wild'), ('Accept', 'Restless and Wild'), ('Accept', 'Restless and Wild')]

Response: 
"Here are 5 popular rock songs for you: 'For Those About To Rock We Salute You' by AC/DC, 'Balls to the Wall' by Accept, and 'Restless and Wild' by Accept. Enjoy!"
